In [25]:
# https://github.com/AlgebraicJulia/Catlab.jl/blob/ce2fde9c63a8aab65cf2a7697f43cd24e5e00b3a/src/theories/Monoidal.jl#L83
cat_rules = [
  :(f ⋅ id(b))    => :f
  :(id(a) ⋅ f)    => :f
  :(f ⋅ (g ⋅ h) ) => :((f ⋅ g) ⋅ h)
 # :(a ⊗ munit())    => :a
  :(munit() ⊗ a)    => :a
  :(a ⊗ (b ⊗ c))  => :((a ⊗ b) ⊗ c)
]

sym_rules = [
  :(σ(a,b) ⋅ σ(b,a)) => :(id(a ⊗ b))
  :(σ(a, b ⊗ c)) => :( (σ(a,b) ⊗ id(c)) ⋅ ( id(b) ⊗ σ(a,c) )) # This rule does not naively type check
  :(σ(a ⊗ b , c)) => :(  ( id(a) ⊗ σ(b,c) ) ⋅ (σ(a,c) ⊗ id(b)))
  :( (f ⊗ h) ⋅ σ(b,d)) => :(  σ(a,c) ⋅ (h ⊗ f )) 
]
# https://github.com/AlgebraicJulia/Catlab.jl/blob/ce2fde9c63a8aab65cf2a7697f43cd24e5e00b3a/src/theories/Monoidal.jl#L127

diag_rules = [
    :( Δ(a) ⋅ (Δ(a) ⊗ id(a))  ) => :( Δ(a) ⋅ (id(a) ⊗ Δ(a)) )
    :( Δ(a) ⋅ (⋄(a) ⊗ id(a))  ) => :( id(a) )
    :( Δ(a) ⋅ (id(a) ⊗ ⋄(a))  ) => :( id(a) )
    :( Δ(a) ⋅ σ(a,a)  ) => :( Δ(a) )

    :( Δ(a ⊗ b)  ) => :( (Δ(a) ⊗ Δ(b))  ⋅ ( id(a) ⊗ σ(a,b) ⊗ id(b) ) )
    :( ⋄(a ⊗ b)  ) => :( ⋄(a) ⊗ ⋄(b) )
    :( Δ(munit())  ) => :( id(munit()) )
    :( ⋄(munit())  ) => :( id(munit()) )
        
]

cart_rules = [
    :( pair(f,k)  ) => :( Δ(a) ⋅ (f ⊗ k) )
    :( proj1(a,b)  ) => :( id(a) ⊗ ⋄(b) )
    :( id(a) ⊗ ⋄(b) ) => :( proj1(a,b)  )
    :( proj2(a,b) ) =>  :(  ⋄(a) ⊗ id(b) )
    :( f ⋅ Δ(b)  ) =>   :(  Δ(a) ⋅ (f ⊗ f)  )
    :( f ⋅ ⋄(b) ) =>    :(  ⋄(b)  )
    
]

name_map = Dict(
    :⋅ => :comp,
    :⊗ => :otimes,
    :Δ => :mcopy,
    :⋄ => :delete,
    :σ => :braid,
    :id => :id,
    :munit => :munit,
    :proj1 => :proj1,
    :proj2 => :proj2,
    :pair => :pair
    
    
)
rules = cat_rules ∪ sym_rules  ∪ diag_rules ∪ cart_rules

to_sexpr(e::Symbol) = haskey(name_map, e) ? name_map[e] : "?$e" 
function to_sexpr(e::Expr)
    "($(join(to_sexpr.(e.args), ' ')))"
end
# rw!("commute-add"; "(+ ?x ?y)" => "(+ ?y ?x)")
egg_rules = []
for e in rules
    #println(to_sexpr(e[1]))
    l = to_sexpr(e[1])
    r = to_sexpr(e[2])
    push!(egg_rules,"rw!( \"$l\" <=> \"$r\" ),")
end
println(join(egg_rules, '\n'))

rw!( "(comp ?f (id ?b))" <=> "?f" ),
rw!( "(comp (id ?a) ?f)" <=> "?f" ),
rw!( "(comp ?f (comp ?g ?h))" <=> "(comp (comp ?f ?g) ?h)" ),
rw!( "(otimes (munit) ?a)" <=> "?a" ),
rw!( "(otimes ?a (otimes ?b ?c))" <=> "(otimes (otimes ?a ?b) ?c)" ),
rw!( "(comp (braid ?a ?b) (braid ?b ?a))" <=> "(id (otimes ?a ?b))" ),
rw!( "(braid ?a (otimes ?b ?c))" <=> "(comp (otimes (braid ?a ?b) (id ?c)) (otimes (id ?b) (braid ?a ?c)))" ),
rw!( "(braid (otimes ?a ?b) ?c)" <=> "(comp (otimes (id ?a) (braid ?b ?c)) (otimes (braid ?a ?c) (id ?b)))" ),
rw!( "(comp (otimes ?f ?h) (braid ?b ?d))" <=> "(comp (braid ?a ?c) (otimes ?h ?f))" ),
rw!( "(comp (mcopy ?a) (otimes (mcopy ?a) (id ?a)))" <=> "(comp (mcopy ?a) (otimes (id ?a) (mcopy ?a)))" ),
rw!( "(comp (mcopy ?a) (otimes (delete ?a) (id ?a)))" <=> "(id ?a)" ),
rw!( "(comp (mcopy ?a) (otimes (id ?a) (delete ?a)))" <=> "(id ?a)" ),
rw!( "(comp (mcopy ?a) (braid ?a ?a))" <=> "(mcopy ?a)" ),
rw!( "(mcopy (otimes ?a ?b))" <=> "(comp (otimes (mcopy ?a) (mcop